In [ ]:
import warnings
from functools import reduce

# filter warning because timestamp shows up in diff-pdf
warnings.filterwarnings('ignore', message='.*SampleCollection contains multiple.*')

In [ ]:
from onecodex.notebooks.report import title, page_break, bibliography, reference

title("Example Report")

# Introduction

This notebook produces all of the plots available in the One Codex client library. It is used for testing.

In [ ]:
## Fetch Data

In [ ]:
from onecodex import Api

ocx = Api()

project = ocx.Projects.get("d53ad03b010542e3")  # get DIABIMMUNE project by ID
samples = ocx.Samples.where(project=project.id, public=True, limit=10)

In [ ]:
page_break()

## Metadata Table

In [ ]:
samples.metadata[[
    "gender",
    "host_age",
    "geo_loc_name",
    "totalige",
    "eggs",
    "vegetables",
    "milk",
    "wheat",
    "rice",
]]

In [ ]:
page_break()

## `plot_metadata`

In [ ]:
samples.plot_metadata(metric='readcount_w_children', rank='phylum')

In [ ]:
samples.plot_metadata(
    plot_type='scatter',
    haxis='Bacteroides',
    facet_by='gender',
    rank="genus"
)

In [ ]:
page_break()

## `plot_bargraph`

In [ ]:
samples.plot_bargraph(return_chart=True) |samples.plot_bargraph(rank='phylum', return_chart=True)

In [ ]:
page_break()

## `plot_heatmap`

In [ ]:
samples.plot_heatmap(metric='abundance_w_children')

In [ ]:
samples.plot_heatmap(rank='phylum', metric='normalized_readcount_w_children')

In [ ]:
samples.plot_heatmap(rank='phylum', metric='readcount_w_children', diversity_metric='unweighted_unifrac')

In [ ]:
page_break()

## `plot_pca`

In [ ]:
samples.plot_pca(color='gender')

In [ ]:
page_break()

## `plot_pcoa`

In [ ]:
samples.plot_pcoa(diversity_metric='unweighted_unifrac', color='gender')

In [ ]:
page_break()

## `plot_mds`

In [ ]:
samples.plot_mds(color='gender')

In [ ]:
page_break()

In [ ]:
reference('test reference 1')
reference('test reference 2')
reference('test reference 3')

bibliography()